# Um momento, por favor!

**Nome: Joana de Medeiros Oliveira Hulse Molinete**

### Introdução:
No presente notebook, busco implementar, treinar e testar o otimizador de Descida do Gradiente com Momento em uma rede neural, utilizando como base o notebook com uma rede neural feita em Python puro disponibilizado pelo professor discente Daniel Cassar. Pretendo, além de aplicar os conhecimentos adquiridos sobre otimização de redes neurais, elucidar o método de funcionamento do Gradient Descent with Momentum e o motivo por trás de sua eficácia na otimização.

### *Gradient Descent* e *Gradient Descent with Momentum*
Na descida de gradiente normal, o gradiente da função de perda é calculado em relação aos parâmetros, e atualiza os parâmetros na direção contrária do gradiente, visando diminuir o erro do modelo. O funcionamento da descida de gradiente segue a seguinte fórmula:

$$
p_{t+1} = p_{t} - \alpha \cdot \nabla L(p_{t})
$$

Onde $p_{t}$ são os parâmetros do modelo, $\alpha$ é a taxa de aprendizado e $\nabla L(p_{t})$ é o gradiente da função de perda.
A descida de gradiente é aplicada no treinamento original do notebook didático utilizado como base do presente notebook.

Já a descida de gradiente com momentum faz o mesmo processo, mas adiciona o fator momentum da física, que basicamente "acumula" uma velocidade a cada nova iteração da rede, fazendo com que o gradiente converja mais rapidamente para seu mínimo, já que tem influência dos gradientes passados a cada nova atualização de parâmetros, e ainda evita falsos mínimos (mínimos locais). A fórmula difere um pouco da tradicional pois adicionamos o fator momento:

$$
v_t = \beta v_{t-1} - \alpha \cdot \nabla J(\theta_t)
$$

$$
\theta_t = \theta_{t-1} + v_t
$$


Onde $\beta$ é o fator *momentum*, $\alpha$ é a taxa de aprendizado, v_t é a velocidade acumulada no tempo _t_, $\nabla J(\theta_t)$ é o gradiente da função de perda e $\theta$ são os parâmetros atualizados.

### Importando as bibliotecas necessárias:

In [1]:
import random
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

semente_aleatoria = 420

C:\Users\joana24003\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### Rede neural:

Como dito anteriormente, a rede neural original foi disponibilizada pelo professor Daniel Cassar [1] e otimizada utilizando o Gradient Descent with Momentum. 

Foi necessário a adição de um novo atributo que armazenasse o histórico de atualizações ("lembrasse o passado") de cada parâmetro conforme a rede neural iterasse durante o treinamento. O novo atributo, chamado de velocidades, está na classe MLP, e inicialmente cria uma lista preenchida com zeros, de acordo com o número de parâmetros da rede neural. Conforme a rede neural trabalha, atribuindo os pesos e viéses em cada neurônio, a lista de parâmetros que inicialmente tinha apenas zeros vai sendo preenchida pelos vetores de velocidade de cada parâmetro a cada atualização.

O conceito de momentum foi implementado no treinamento da rede, na parte de atualização de parâmetros, utilizando a seguinte fórmula:

$$v_t = \beta v_{t-1} - \alpha \nabla J(\theta_t)$$
$$\theta_t = \theta_{t-1} + v_t$$


In [2]:
def calcular_rmse(y_true, y_pred):
    n = len(y_true)
    soma_erro = 0
    
    for i in range(n):
        erro = y_pred[i] - y_true[i]
        erro_quadrado = erro ** 2
        soma_erro += erro_quadrado

    mse = soma_erro / n
    rmse = mse ** (1/2)
    return rmse


class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    def sig(self):
        """Realiza a operação: exp(self) / (exp(self) + 1)"""
        return self.exp() / (self.exp() + 1)
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

            
class Neuronio:
    def __init__(self, num_dados_entrada):
        self.vies = Valor(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))
            
    def __call__(self, x):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        dado_de_saida = soma.sig()
        
        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]
    
    
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params
    
    
class MLP:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)
            
        self.camadas = camadas
        
        # novo atributo!
        self.velocidades = [0 for _ in self.parametros()] # precisamos que o momentum seja uma lista de zeros do mesmo tamanho que os parametros
        
    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params

## Testando a rede neural com o dataset Boston Housing:

Para que seja possível observar se a implementação do otimizador Gradient Descent with Momentum foi efetiva, vamos testar a rede neural em um dataset. Aqui, o objetivo da nossa rede neural é prever o preço das casas. 

In [3]:
url = 'https://raw.githubusercontent.com/selva86/datasets/refs/heads/master/BostonHousing.csv'

df = pd.read_csv(url)

df

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


### Separando os dados:

Vamos separar nosso dataset entre os dados de treino e de teste. Aqui, separamos 80% dos dados para treino, e 20% dos dados para teste. Além disso, as features utilizadas foram "indus" (proporção de acres destinados a negócios não comerciais), "rm" (número médio de cômodos por residência), "crim" (taxa de criminalidade per capita por cidade) e "lstat" (percentual da população de nível socioeconômico baixo), enquanto nosso target foi "medv", que é o valor médio do imóvel.

In [4]:
tamanho_teste = 0.2

FEATURES = ['indus', 'rm', 'crim', 'lstat']
TARGET = ['medv']

In [5]:
indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size = tamanho_teste, random_state = semente_aleatoria
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()

X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

Normalizando os dados:

In [6]:
normalizador = StandardScaler()

X_treino_norm = normalizador.fit_transform(X_treino)
X_teste_norm = normalizador.fit_transform(X_teste)

y_treino_norm = normalizador.fit_transform(y_treino.reshape(-1,1))
y_teste_norm = normalizador.fit_transform(y_teste.reshape(-1,1))

print(X_treino)
print()
print(X_treino_norm)

[[ 4.05     6.416    0.09178  9.04   ]
 [ 3.41     6.417    0.04684  8.81   ]
 [ 3.97     6.842    0.65665  6.9    ]
 ...
 [18.1      5.713    6.96215 17.11   ]
 [10.81     6.065    0.09164  5.52   ]
 [18.1      6.436    5.58107 16.22   ]]

[[-1.04197113  0.1916685  -0.4466585  -0.50585907]
 [-1.13864686  0.19307859 -0.45277542 -0.53807131]
 [-1.05405559  0.79236987 -0.36977232 -0.80557297]
 ...
 [ 1.08036324 -0.79962978  0.48848856  0.62437044]
 [-0.02083374 -0.30327559 -0.44667755 -0.99884642]
 [ 1.08036324  0.21987044  0.30050554  0.49972307]]


Definimos a arquitetura da rede neural tendo como entrada 4 valores ("INDUS", "RM", "CRIM", "LSTAT") e contendo 2 camadas ocultas, a primeira com 3 neurônios e a última com 2 neurônios. A saída é um único valor, porque é uma rede regressora que busca estimar um valor depois do treinamento.

In [7]:
NUM_DADOS_DE_ENTRADA = 4
NUM_DADOS_DE_SAIDA = 1    
CAMADAS_OCULTAS = [3, 2]  

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

minha_mlp_GDM = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede)

### Treinando a rede neural:

Aqui, introduzimos efetivamente a fórmula do Gradient Descent with Momentum, na etapa de atualização de parâmetros.

In [8]:
NUM_EPOCAS = 200
TAXA_DE_APRENDIZADO = 0.01
fator_momentum = 0.9

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in X_treino_norm:
        previsao = minha_mlp_GDM(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_treino_norm, y_pred):
        residuo = yp - yt
        erro_quadratico = residuo ** 2
        erros.append(erro_quadratico)        
    loss = sum(erros)/len(erros)

    # zero grad
    for p in minha_mlp_GDM.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros com momentum
    for indice, parametro in enumerate(minha_mlp_GDM.parametros()):
        v_antiga = minha_mlp_GDM.velocidades[indice]
        grad = parametro.grad
        nova_v = fator_momentum * v_antiga - TAXA_DE_APRENDIZADO * grad
        minha_mlp_GDM.velocidades[indice] = nova_v 
        parametro.data += nova_v

    # mostra resultado (opcional)
    if epoca % 10 == 0:
        print(f'Época: {epoca}, Loss: {loss.data}')

Época: 0, Loss: [1.42153507]


C:\Users\joana24003\AppData\Local\Temp\ipykernel_30264\673356561.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data = math.exp(self.data)


Época: 10, Loss: [1.34772348]
Época: 20, Loss: [1.2258148]
Época: 30, Loss: [1.13138805]
Época: 40, Loss: [1.07825621]
Época: 50, Loss: [1.05104413]
Época: 60, Loss: [1.03653035]
Época: 70, Loss: [1.02806284]
Época: 80, Loss: [1.02264163]
Época: 90, Loss: [1.01888691]
Época: 100, Loss: [1.01612346]
Época: 110, Loss: [1.01399525]
Época: 120, Loss: [1.01229995]
Época: 130, Loss: [1.01091438]
Época: 140, Loss: [1.00975895]
Época: 150, Loss: [1.00877964]
Época: 160, Loss: [1.00793835]
Época: 170, Loss: [1.00720733]
Época: 180, Loss: [1.00656582]
Época: 190, Loss: [1.00599798]


### Testando a previsão da rede:

Vejamos então como nossa rede neural se sai ao tentar prever valores reais do nosso dataset.

In [9]:
y_pred_GDM = []

for exemplo in X_teste_norm:
    previsao = minha_mlp_GDM(exemplo)
    y_pred_GDM.append(previsao)

print(y_pred_GDM)
print()

RMSE_GDM = calcular_rmse(y_teste_norm, y_pred_GDM)
print(f"RMSE com o otimizador Gradient Descent with Momentum: {RMSE_GDM}")

C:\Users\joana24003\AppData\Local\Temp\ipykernel_30264\673356561.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data = math.exp(self.data)


[Valor(data=0.08963313241634757), Valor(data=0.09619815892240789), Valor(data=0.09784902209493065), Valor(data=0.0914494959242233), Valor(data=0.08998535919608203), Valor(data=0.0902168147522078), Valor(data=0.0911429633959693), Valor(data=0.09151739821002777), Valor(data=0.0917158539540397), Valor(data=0.09044409211611965), Valor(data=0.09237911958135923), Valor(data=0.09750681112023592), Valor(data=0.09422857387587706), Valor(data=0.09754678044073369), Valor(data=0.09164647842053406), Valor(data=0.09087862748062209), Valor(data=0.09036678325520028), Valor(data=0.09761214288812993), Valor(data=0.09748310252988097), Valor(data=0.09386267794513442), Valor(data=0.09223615827951856), Valor(data=0.09128825247520884), Valor(data=0.0924752201667965), Valor(data=0.09166287726979168), Valor(data=0.09208991806346445), Valor(data=0.09062896545659287), Valor(data=0.0919399176778554), Valor(data=0.09156226986678297), Valor(data=0.09689854538026685), Valor(data=0.09148437994055285), Valor(data=0.09

### Treinando e testando a rede com Gradient Descent tradicional:

Para fins de comparação, vamos treinar e testar uma rede neural otimizada com o Gradient Descent tradicional.

In [10]:
minha_mlp_GD = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede)

NUM_EPOCAS = 200
TAXA_DE_APRENDIZADO = 0.01

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in X_treino_norm:
        previsao = minha_mlp_GD(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_treino_norm, y_pred):
        residuo = yp - yt
        erro_quadratico = residuo ** 2
        erros.append(erro_quadratico)        
    loss = sum(erros)

    # zero grad
    for p in minha_mlp_GD.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in minha_mlp_GD.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    # mostra resultado (opcional)
    if epoca % 10 == 0:
        print(f'Época: {epoca}, Loss: {loss.data}')

Época: 0, Loss: [438.44879373]


C:\Users\joana24003\AppData\Local\Temp\ipykernel_30264\673356561.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data = math.exp(self.data)


Época: 10, Loss: [402.72700865]
Época: 20, Loss: [397.67145927]
Época: 30, Loss: [380.69426793]
Época: 40, Loss: [332.11754703]
Época: 50, Loss: [288.73405582]
Época: 60, Loss: [270.46231847]
Época: 70, Loss: [262.85394315]
Época: 80, Loss: [259.0111648]
Época: 90, Loss: [256.79006961]
Época: 100, Loss: [255.388203]
Época: 110, Loss: [254.44299044]
Época: 120, Loss: [253.7718194]
Época: 130, Loss: [253.27504392]
Época: 140, Loss: [252.89465531]
Época: 150, Loss: [252.59502431]
Época: 160, Loss: [252.35326541]
Época: 170, Loss: [252.15411693]
Época: 180, Loss: [251.98707504]
Época: 190, Loss: [251.84471512]


In [11]:
y_pred_GD = []

for exemplo in X_teste_norm:
    previsao = minha_mlp_GD(exemplo)
    y_pred_GD.append(previsao)

print(y_pred_GD)
print()

RMSE_GD = calcular_rmse(y_teste_norm, y_pred_GD)
print("RMSE com o otimizador Gradient Descent:", RMSE_GD)

C:\Users\joana24003\AppData\Local\Temp\ipykernel_30264\673356561.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data = math.exp(self.data)


[Valor(data=0.00194779028910585), Valor(data=0.02383890000942694), Valor(data=0.9468009029251074), Valor(data=0.0022246438574541204), Valor(data=0.0018829305399597836), Valor(data=0.002612977982199203), Valor(data=0.0019733680359636263), Valor(data=0.002672280931297396), Valor(data=0.0029747240831131828), Valor(data=0.0020855339789041197), Valor(data=0.5481035052921844), Valor(data=0.9528967880000554), Valor(data=0.002284699257194837), Valor(data=0.6232170258677272), Valor(data=0.0028249715556526254), Valor(data=0.0019162224985720285), Valor(data=0.001879515303522894), Valor(data=0.08369343061869117), Valor(data=0.010031842677115406), Valor(data=0.015077803941561815), Valor(data=0.008684774520695328), Valor(data=0.002151254832048131), Valor(data=0.46489397120756476), Valor(data=0.002860045425330443), Valor(data=0.0028136073199950123), Valor(data=0.0020346526661393885), Valor(data=0.00618513323849509), Valor(data=0.0022499182247429626), Valor(data=0.03121910830906634), Valor(data=0.0029

## Conclusões:

Após o treinamento das duas redes neurais com o mesmo dataset, e seu teste, podemos notar que o modelo otimizado com o Gradient Descent tradicional obteve um desempenho melhor do que o algoritmo otimizado com o Gradient Descent com o fator Momentum, o que foge um pouco do esperado. 

No entanto, observamos que durante o treinamento, a rede com GDM apresentou uma loss final ( ~ 1.005) menor em comparação com a GD tradicional ( ~ 251.844). Podemos inferir que a rede treinada com GDM convergiu para um mínimo mais estável durante o treinamento, porém no teste a rede treinada com GD generalizou melhor os dados, o que explica o menor RMSE da GD tradicional ( ~ 0.768) quando comparado com o RMSE do GDM ( ~ 1.002).

## Refêrencias:

[1] Momentum-based Gradient Optimizer - ML. Geeks for geeks, 2025. Disponível em: https://www.geeksforgeeks.org/ml-momentum-based-gradient-optimizer-introduction/

[2] BROWNLEE, Jason. Gradient Descent With Momentum from Scratch. Machine Learning Mastery. Disponível em: https://machinelearningmastery.com/gradient-descent-with-momentum-from-scratch/

[3] MAHDID, Yacine. Stochastic Gradient Descent with Momentum in Python. Disponível em: https://www.youtube.com/watch?v=7EuiXb6hFAM